In [1]:
from docling.document_converter import DocumentConverter
import requests
import re
from typing import List, Dict

In [16]:
def scrape_article_text(url: str) -> str:
    """
    Fetch the article at `url` and extract its full text using Docling's DocumentConverter.
    """
    response = requests.get(url)
    converter = DocumentConverter()
    result = converter.convert(response.text)
    # Export document to Markdown (body text)
    content = result.document.export_to_markdown()
    return content

test_url = 'https://news.google.com/rss/articles/CBMi7AFBVV95cUxQa0lBUXo4THktUVBNQ3JTZ0d3dWpwWEJuc3EzaEFVbndtX1JwY1NVOEZ2U3pBMl9zYUJRdU1WOTN5bGxMdG02OEdzZjZvOU9ieVY4eHV3X2Z1X2FWdmRjWC1VSkZfMFRHOWRjcUdwZDJ3REFZMzBTZjVmY2pEM1BRNjZqS3hWYmJwWEJBLWhZV1hPTTQyYWY2UWk4d2ZmdDJWNXJPT2VXOEkzempKLXphaGpvVHBESEVFaEV5VkdzOHRzdWl3RkhMMFMxOW9TNWQxamlCZXVnZFNSQVRTdENkem9OYWFCLXNta0dqeQ?oc=5'


In [21]:
import re
import requests
from bs4 import BeautifulSoup
from typing import Optional

def resolve_google_rss_url(wrapper_url: str, timeout: float = 5.0) -> str:
    """
    Given a Google News RSS‐style URL (e.g. 'https://news.google.com/rss/articles/CBM...'),
    fetch the HTML, look for a <meta http-equiv="refresh" content="0;URL=REAL_URL"> or
    a JS snippet window.location="REAL_URL", and return REAL_URL. If nothing is found,
    return the original wrapper_url.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/114.0.0.0 Safari/537.36"
    }
    try:
        resp = requests.get(wrapper_url, headers=headers, timeout=timeout)
        resp.raise_for_status()
    except Exception:
        # If the request fails, just return the original link
        return wrapper_url

    html = resp.text
    soup = BeautifulSoup(html, "html.parser")

    # 1) Look for <meta http-equiv="refresh" content="0; URL=REAL_URL">
    meta = soup.find("meta", attrs={"http-equiv": "refresh"})
    if meta and "content" in meta.attrs:
        content = meta["content"]  # e.g. '0; url=https://www.datacenter...'
        # Extract everything after "url="
        match = re.search(r'url=(.+)', content, flags=re.IGNORECASE)
        if match:
            return match.group(1).strip().strip('"').strip("'")

    # 2) If no meta-refresh, search for a JS snippet: window.location = "REAL_URL"
    #    Sometimes Google News uses: <script>window.location.href="REAL_URL"</script>
    js_redirect = re.search(
        r'window\.location(?:\.href)?\s*=\s*["\']([^"\']+)["\']',
        html
    )
    if js_redirect:
        return js_redirect.group(1).strip()

    # 3) If neither is found, fall back to the original
    return wrapper_url

# --------------------------
# Example usage:
if __name__ == "__main__":
    wrapper = (
        "https://news.google.com/rss/articles/CBMi7AFBVV95cUxQa0lBUXo4THktUVBNQ3JTZ0d3dWpwWEJuc3EzaEFVbndt"
        "X1JwY1NVOEZ2U3pBMl9zYUJRdU1WOTN5bGxMdG02OEdzZjZvOU9ieVY4eHV3X2Z1X2FWdmRjWC1VSkZfMFRHOWRjcUdwZDJ3"
        "REFZMzBTZjVmY2pEM1BRNjZqS3hWYmJwWEJBLWhZV1hPTTQyYWY2UWk4d2ZmdDJWNXJPT2VXOEkzempKLXphaGpvVHBESEVF"
        "aEV5VkdzOHRzdWl3RkhMMFMxOW9TNWQxamlCZXVnZFNSQVRTdENkem9OYWFCLXNta0dqeQ?oc=5"
    )
    real = resolve_google_rss_url(wrapper)
    print("Resolved URL:", real)


Resolved URL: https://news.google.com/rss/articles/CBMi7AFBVV95cUxQa0lBUXo4THktUVBNQ3JTZ0d3dWpwWEJuc3EzaEFVbndtX1JwY1NVOEZ2U3pBMl9zYUJRdU1WOTN5bGxMdG02OEdzZjZvOU9ieVY4eHV3X2Z1X2FWdmRjWC1VSkZfMFRHOWRjcUdwZDJ3REFZMzBTZjVmY2pEM1BRNjZqS3hWYmJwWEJBLWhZV1hPTTQyYWY2UWk4d2ZmdDJWNXJPT2VXOEkzempKLXphaGpvVHBESEVFaEV5VkdzOHRzdWl3RkhMMFMxOW9TNWQxamlCZXVnZFNSQVRTdENkem9OYWFCLXNta0dqeQ?oc=5


In [33]:

import os
import json
from typing import List, Dict
from datetime import datetime, timezone
from dotenv import load_dotenv

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

True

In [34]:
test_items = [
    {
        "title": "Bitfarms forçada a interromper mineração de criptomoedas em seu data center na Argentina",
        "source": "Data Center Dynamics",
        "url": "https://www.datacenterdynamics.com/br/not%C3%ADcias/bitfarms-for%C3%A7ada-a-interromper-minera%C3%A7%C3%A3o-de-criptomoedas-em-seu-data-center-na-argentina/",
        "pubDate": datetime.now(timezone.utc),
    },
    {
        "title": "Evento sobre IA no Brasil",
        "source": "Tech News Brasil",
        "url": "https://example.com/ia-brasil-evento",
        "pubDate": datetime.now(timezone.utc),
    },
]

In [35]:
CLASSIFICATION_PROMPT_TEMPLATE = """

You are a specialized news‐classification LLM. For each incoming news article, you will receive exactly two fields:
- title
- url

Your ONLY output must be a single, valid JSON object with exactly two keys:
  • {{"label"}}: either "relevant" or "irrelevant"
  • {{"category"}}: one of "clientes", "competidores", "governo", "inovação", "outros", or "" (the empty string). 
      If {{"label"}} is "irrelevant", then {{"category"}} must be "".

You must NOT output any extra text—no explanations, no punctuation, no comments, no "ok," nothing. Strictly produce exactly one JSON object per article.

---

### 1. Industry Context

1. A hyperscale datacenter company in Brazil monitors global and local news that could impact its strategic planning, expansion, investment decisions, or risk management.  
2. Weekly clippings are curated into five sections—CLIENTES (Clients), COMPETIDORES (Competitors), GOVERNO (Government), INOVAÇÃO (Innovation), and OUTROS (Other)—with only those items that meet strict inclusion criteria.  
3. The overarching goal is to capture market‐moving information (e.g., new build announcements, regulatory changes, major technology rollouts) rather than routine or tangential news.

---

### 2. Relevance Categories & Definitions

#### 2.1 Clientes (Cloud Providers)  
- **Include**: Announcements by or about major global cloud providers (AWS, Azure, Google Cloud, Alibaba Cloud, Huawei Cloud, etc.) when they signal infrastructure expansions, new data center regions, large CAPEX projects, or strategic shifts (e.g., decommissioning, regional refocusing).  
  • *Exemplo:* "Amazon vai investir mais de US\$ 4 bilhões em data centers no Chile" (AWS expansion, new region).  
  • *Exemplo:* "Alibaba Cloud lança Qwen3 e acelera corrida pela liderança em IA generativa na China" (rebuild de infraestrutura para IA, CAPEX de ¥380 bi).  
  • *Exemplo:* "Google Cloud registra trimestre forte e receita de 28% em relação ao ano anterior" (sinaliza maior capex e foco estratégico no Brasil).  
- **Exclude**:  
  • End‐user migrations ou anúncios de clientes sem detalhe de infraestrutura.  
    – "Pepsi switches to Alibaba Cloud" → irrelevante.  
    – "Magalu vai trazer a Alibaba Cloud para o Brasil?" → irrelevante (sem detalhes de build/region).

#### 2.2 Competidores (Primarily Brazilian Operators)  
- **Include**: Notícias sobre operadores de data center brasileiros ou latinos—Ascenty, Scala, Equinix, Odata, Elea, HostDime, Claranet, NextStream, RT‐One, etc.—que afetem capacidade, financiamento, novos builds, grandes expansões ou M&A.  
  • *Exemplo:* "Empresa global investe R\$ 250 milhões em novo data center na Paraíba" (HostDime).  
  • *Exemplo:* "Ascenty realiza Press Class sobre data centers" (evento de operador brasileiro).  
  • *Exemplo:* "Scala Data Centers assina contrato para compra de energia renovável" (PPA da Scala).  
- **Exclude**:  
  • Relatórios financeiros rotineiros (quarterly earnings) a menos que revelem investimento de capital ou mudança de capacidade significativa.  
    – "Elea Data Centers cresce 8% e projeta faturar R\$ 200 milhões" → relevante (sinaliza trajetória de crescimento).  
    – "Elea Data Centers reports Q1 earnings summary" sem detalhes sobre novo capex → irrelevante.

#### 2.3 Governo (Regulatory & Policy Decisions)  
- **Include**: Decisões federais ou estaduais que impactem diretamente instalação, operação, tributação, regulação ou incentivos para data centers. Exemplos: medida provisória para desoneração, licenciamento de utilidade pública para energia ou conectividade, legislação específica para data centers.  
  • *Exemplo:* "MP de desoneração de data centers deve sair nesta semana, diz Haddad" (incentivo fiscal).  
  • *Exemplo:* "Governo brasileiro cria grupo para coordenar plano de IA" (funding de infraestrutura via PBIA).  
  • *Exemplo:* "Ceará abre licitação para contratar rede neutra" (afeta conectividade para DCs).  
- **Exclude**:  
  • Políticas tecnológicas genéricas sem menção a data centers (e.g. "5G rollout" sem referência a conectividade DC).  
  • Comunicados locais de governo sobre IA sem infraestrutura DC mencionada.

#### 2.4 Inovação (Technology & Infrastructure Advances)  
- **Include**: Novos hardwares, soluções de resfriamento, arquiteturas de energia ou tecnologias emergentes que possam mudar eficiência, custo ou design de datacenters. Exemplos: chips de computação quântica, resfriamento líquido, sistemas HVDC, aceleradores de IA, soluções renováveis dedicadas a DCs.  
  • *Exemplo:* "Google instala 100 milhões de células de íons de lítio em seus data centers" (UPS Li‐ion).  
  • *Exemplo:* "Nvidia working with data center partners to build 800V HVDC power systems" (HVDC para eficiência).  
  • *Exemplo:* "Vertiv launches modular, prefabricated overhead infrastructure system SmartRow" (infraestrutura modular).  
- **Exclude**:  
  • Inovações de consumo sem aplicação explícita a data centers (e.g. novos chips de smartphone).  
  • Anúncios genéricos de modelos de IA sem menção a exigências de infraestrutura DC.

#### 2.5 Outros (Miscellaneous, but Directly Impactful)  
- **Include**:  
  1. Macro‐tendências que indiretamente afetam economia ou demanda de data centers, risco ou custos—e.g., mudanças significativas no preço da energia, previsões de mercado, avisos sobre sobre‐oferta/infrautilização.  
     • *Exemplo:* "Moody's alerta para risco de bolha em data centers de IA" (aviso de sobrecapacidade e curva de custo energético).  
  2. Dados de penetração de banda larga que constituam marco relevante para demanda de computação no Brasil.  
     • *Exemplo:* "93% dos domicílios brasileiros têm banda larga fixa" (milestone que sinaliza aumento de demanda DC).  
  3. Anúncios de cabos submarinos ou rede óptica que mudem drasticamente conectividade ou latência na LatAm.  
     • *Exemplo:* "Meta lança cabo submarino com passagem pelo Nordeste" (impactos na latência para DCs no Brasil).  
  4. Relatórios de mercado projetando consumo futuro de energia, pegada de carbono, ou restrições de recursos para DCs (e.g., IEA, whitepapers).  
  5. Mandatos de ESG/sustentabilidade para grandes data centers—e.g., novos PPAs renováveis, regulamentações de uso de água para resfriamento.  
- **Exclude**:  
  • Notícias econômicas gerais sem ligação explícita a carga de trabalho DC, conectividade ou custos de energia.  
  • Anúncios de telecom ou consumo que não sejam um ponto de inflexão relevante para conectividade DC (e.g., rollout de estação 5G sem menção a tráfego DC).  
  • Artigos que mencionem DCs ou nuvem de passagem, sem insight acionável para planejamento de capacidade, design de rede, orçamento energético ou mitigação de risco.

---

### 3. Inclusion Criteria (What Makes an Article "Relevant")

Para classificar como **relevant**, o artigo deve satisfazer ambos:
1. **Impacto Direto** em ao menos uma das cinco categorias (Clientes, Competidores, Governo, Inovação, Outros).  
2. **Escopo Material** para estratégia de um hyperscale datacenter no Brasil/LatAm—i.e., afetar capacidade, estrutura de custos, ambiente regulatório, roadmap tecnológico.

> **Exemplos-citação**:  
> • Clientes: "Amazon vai investir mais de US\$ 4 bilhões em data centers no Chile" → detalha escopo de três AZs, capex de 4 bi USD.  
> • Competidores: "Ascenty realiza Press Class sobre data centers, destacando investimentos de R\$ 10 bilhões" → sinaliza expansão de competidor e possível mudança de market share.  
> • Governo: "MP de desoneração de data centers deve sair nesta semana, diz Haddad" → grande incentivo fiscal para novos builds no Brasil.  
> • Inovação: "Nvidia working with data center partners to build 800V HVDC power systems" → tecnologia HVDC com impacto direto em eficiência e PUE.  
> • Outros: "93% dos domicílios brasileiros têm banda larga fixa" → marco de penetração que afeta previsões de demanda computacional e uso de DC.

---

### 4. Exclusion Criteria (What Makes an Article "Irrelevant")

Classifique como **irrelevant** se qualquer uma das seguintes condições se aplica:
1. Nenhuma ligação direta a pelo menos uma das cinco categorias.  
2. Foco em adoção de nuvem/serviços por usuário final não‐hyperscale (e.g., "Pepsi migrates to Alibaba Cloud" → irrelevante).  
3. Atualização rotineira ou menor (quarterly earnings sem novo capex ou mudança de capacidade significativa).  
4. Setores de consumo ou não‐DC (e.g., lançamento de smartphone, e‐commerce sem referência a DC).  
5. Falta de qualquer insight acionável para estratégia de um hyperscale DC (sem menção a capacidade, regulação, inovação, ou risco macro).

> **Exemplos‐irrelevantes**:  
> • "Magalu vai trazer a Alibaba Cloud para o Brasil?" → especulação sem detalhes de infraestrutura.  
> • "Feira de tecnologia discute metaverso" → evento sem foco em infraestrutura DC.  
> • "93% dos domicílios brasileiros têm Netflix" → estatística de streaming sem impacto de conectividade DC.

---

### 5. Illustrative Examples

Apenas para referência interna—não inclua explicações ao classificar. Use-os para calibrar:

1. **Example A**  
   Title: "AWS integrates DeepSeek‐R1 into Amazon Bedrock"  
   → Categoria: clientes (relevant)  
   Rationale: Sinaliza AWS expandindo oferta de IA, impactando demanda DC.

2. **Example B**  
   Title: "Pepsi switches to Alibaba Cloud for marketing analytics"  
   → Categoria: "" (irrelevant)  
   Rationale: Apenas usuário final, sem anúncio de build ou infraestrutura.

3. **Example C**  
   Title: "LatAm datacenter firms reassessing workforce size and outsourcing"  
   → Categoria: competidores (relevant)  
   Rationale: Aborda operadores latinos ajustando custos e estratégia operacional.

4. **Example D**  
   Title: "Itaú launches institute for quantum computing research"  
   → Categoria: "" (irrelevant)  
   Rationale: R&D bancário sem menção a nova infraestrutura DC.

5. **Example E**  
   Title: "93% dos domicílios brasileiros têm banda larga fixa"  
   → Categoria: outros (relevant)  
   Rationale: Marco de penetração de banda larga diretamente ligado a demanda DC.

6. **Example F**  
   Title: "Meta lança cabo submarino com passagem pelo Nordeste"  
   → Categoria: outros (relevant)  
   Rationale: Novo cabo muda latência e influência localização de DCs no Brasil.

---

### 6. Output Requirements

**FINAL REMINDER**:  
- Output EXACTLY one JSON object with keys {{"label"}} e {{"category"}}.  
- {{"label"}} = "relevant" ou "irrelevant".  
- {{"category"}} = "clientes", "competidores", "governo", "inovação", "outros", ou "" se {{"label"}} for "irrelevant".  
- Não emita texto adicional fora do JSON.

**Formato EXATO**:  

{{"label": "relevant", "category": "competidores"}} [caso seja relevante]

ou

{{"label": "irrelevant", "category": ""}} [caso seja irrelevante]


"""

[{'title': 'Bitfarms forçada a interromper mineração de criptomoedas em seu data center na Argentina',
  'source': 'Data Center Dynamics',
  'url': 'https://www.datacenterdynamics.com/br/not%C3%ADcias/bitfarms-for%C3%A7ada-a-interromper-minera%C3%A7%C3%A3o-de-criptomoedas-em-seu-data-center-na-argentina/',
  'pubDate': datetime.datetime(2025, 6, 1, 18, 59, 36, 629961, tzinfo=datetime.timezone.utc),
  'class': 'relevant',
  'category': 'competidores'},
 {'title': 'Evento sobre IA no Brasil',
  'source': 'Tech News Brasil',
  'url': 'https://example.com/ia-brasil-evento',
  'pubDate': datetime.datetime(2025, 6, 1, 18, 59, 36, 629966, tzinfo=datetime.timezone.utc),
  'class': 'irrelevant',
  'category': ''}]